# Projeto 1 - Objetos

Disciplina SCC0250 - Computação Gráfica

<hr>

João Pedro Ribeiro da Silva - 12563727

Miller Matheus Lima Anacleto Rocha - 13727954

Código baseado naquele desenvolvido e disponibilizado pelo professor na Aula 09 - Prática sobre Model e View 

## Bibliotecas

In [1]:
import glfw
from OpenGL.GL import *
import numpy as np
import glm
import math
from numpy import random

from shader_s import Shader

## Inicializando janela

In [2]:
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE)

altura = 700
largura = 700

window = glfw.create_window(largura, altura, "Cabana assombrada", None, None)

if (window == None):
    print("Failed to create GLFW window")
    glfwTerminate()
    
glfw.make_context_current(window)

## Constroi e compila os shaders. Também "linka" eles ao programa

#### Novidade aqui: modularização dessa parte do código --- temos agora uma classe e arquivos próprios para os shaders (vs e fs)
Créditos: https://learnopengl.com

In [3]:
ourShader = Shader("vertex_shader.vs", "fragment_shader.fs")
ourShader.use()

program = ourShader.getProgram()

## Preparando dados para enviar a GPU

Até aqui, compilamos nossos Shaders para que a GPU possa processá-los.

Por outro lado, as informações de vértices geralmente estão na CPU e devem ser transmitidas para a GPU.


### Carregando Modelos (vértices e texturas) a partir de Arquivos

A função abaixo carrega modelos a partir de arquivos no formato WaveFront (.obj).

Para saber mais sobre o modelo, acesse: https://en.wikipedia.org/wiki/Wavefront_.obj_file

In [4]:
VERTICES_LIST = []

In [5]:
def load_model_from_file(filename):
    """Loads a Wavefront OBJ file. """
    objects = {}
    vertices = []
    faces = []

    # abre o arquivo obj para leitura
    for line in open(filename, "r"): ## para cada linha do arquivo .obj
        if line.startswith('#'): continue ## ignora comentarios
        values = line.split() # quebra a linha por espaço
        if not values: continue

        ### recuperando vertices
        if values[0] == 'v':
            vertices.append(values[1:4])

        ### recuperando faces 
        elif values[0] == 'f':
            face = [int(v) for v in values[1:]]
            faces.append(face)

    model = {}
    model['vertices'] = vertices
    model['faces'] = faces

    return model

def load_obj(objFile):
    global VERTICES_LIST
    modelo = load_model_from_file(objFile)
    
    ### inserindo vertices do modelo no vetor de vertices
    verticeInicial = len(VERTICES_LIST)
    print('Processando modelo {}. Vertice inicial: {}'.format(objFile, verticeInicial))
    for face in modelo['faces']:
        VERTICES_LIST.extend([modelo['vertices'][f-1] for f in face])
        
    verticeFinal = len(VERTICES_LIST)
    print('Processando modelo {}. Vertice final: {}'.format(objFile, verticeFinal))
    
    return verticeInicial, verticeFinal - verticeInicial

### Vamos carregar cada modelo e definir funções para desenhá-los

In [6]:
# carrega objetos
vertice_inicial_fantasma, num_vertices_fantasma = load_obj('objetos/fantasma.obj')
arvores = []
NUM_ARVORES = 7
for i in range(NUM_ARVORES):
    vertice_inicial_arvore, num_vertices_arvore = load_obj('objetos/arvore.obj')
    arvores.append({
        'vertice_inicial': vertice_inicial_arvore, 
        'num_vertices': num_vertices_arvore
    })
vertice_inicial_grama, num_vertices_grama = load_obj('objetos/grama.obj')
vertice_inicial_lua, num_vertices_lua = load_obj('objetos/lua.obj')
vertice_inicial_cabana, num_vertices_cabana = load_obj('objetos/cabana.obj')
vertice_inicial_caderno, num_vertices_caderno = load_obj('objetos/caderno.obj')

def model_objeto(vertice_inicial, num_vertices, t_x=0, t_y=0, t_z=0, s_x=1, s_y=1, s_z=1, r_x=0, r_y=0, r_z=0):
    # aplica a matriz model
    
    mat_model = model(t_x, t_y, t_z,  # translação
                      s_x, s_y, s_z,  # escala
                      r_x, r_y, r_z)  # rotação
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)

def desenha_objeto(vertice_inicial, num_vertices, cor=[1,1,1,1]):
    # cor
    loc_color = glGetUniformLocation(program, "color")
    glUniform4f(loc_color, cor[0], cor[1], cor[2], cor[3])
       
    # desenha o objeto
    glDrawArrays(GL_TRIANGLES, vertice_inicial, num_vertices) ## renderizando

Processando modelo objetos/fantasma.obj. Vertice inicial: 0
Processando modelo objetos/fantasma.obj. Vertice final: 1338
Processando modelo objetos/arvore.obj. Vertice inicial: 1338
Processando modelo objetos/arvore.obj. Vertice final: 1518
Processando modelo objetos/arvore.obj. Vertice inicial: 1518
Processando modelo objetos/arvore.obj. Vertice final: 1698
Processando modelo objetos/arvore.obj. Vertice inicial: 1698
Processando modelo objetos/arvore.obj. Vertice final: 1878
Processando modelo objetos/arvore.obj. Vertice inicial: 1878
Processando modelo objetos/arvore.obj. Vertice final: 2058
Processando modelo objetos/arvore.obj. Vertice inicial: 2058
Processando modelo objetos/arvore.obj. Vertice final: 2238
Processando modelo objetos/arvore.obj. Vertice inicial: 2238
Processando modelo objetos/arvore.obj. Vertice final: 2418
Processando modelo objetos/arvore.obj. Vertice inicial: 2418
Processando modelo objetos/arvore.obj. Vertice final: 2598
Processando modelo objetos/grama.obj. V

### Para enviar nossos dados da CPU para a GPU, precisamos requisitar dois slots (buffers): um para os vértices e outro para as texturas.

In [7]:
buffer_VBO = glGenBuffers(1)

### Enviando coordenadas de vértices para a GPU

Veja os parâmetros da função glBufferData [https://www.khronos.org/registry/OpenGL-Refpages/gl4/html/glBufferData.xhtml]

In [8]:
vertices = np.zeros(len(VERTICES_LIST), [("position", np.float32, 3)])
vertices['position'] = VERTICES_LIST


# Upload data
glBindBuffer(GL_ARRAY_BUFFER, buffer_VBO)
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_STATIC_DRAW)
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)
loc_vertices = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc_vertices)
glVertexAttribPointer(loc_vertices, 3, GL_FLOAT, False, stride, offset)

## Câmera fixa

Matrizes View e Projection são constantes

In [9]:
# view
cameraPos   = glm.vec3(0.0, 0.0, -10.0)
cameraFront = glm.vec3(0.0, 0.0, 1.0)
cameraUp    = glm.vec3(0.0, 1.0, 0.0)

MAT_VIEW = np.array(
    glm.lookAt(cameraPos, cameraPos + cameraFront, cameraUp)
)
loc_view = glGetUniformLocation(program, "view")
glUniformMatrix4fv(loc_view, 1, GL_TRUE, MAT_VIEW)

# projection
fov   =  45.0

MAT_PROJECTION = np.array(
    glm.perspective(glm.radians(fov), largura/altura, 0.1, 100.0)
)
loc_projection = glGetUniformLocation(program, "projection")
glUniformMatrix4fv(loc_projection, 1, GL_TRUE, MAT_PROJECTION)   

## Matriz Model

In [10]:
def model(t_x=0, t_y=0, t_z=0, s_x=1, s_y=1, s_z=1, r_x=0, r_y=0, r_z=0):
    
    matrix_transform = glm.mat4(1.0) # instanciando uma matriz identidade
       
    # aplicando translacao (terceira operação a ser executada)
    matrix_transform = glm.translate(matrix_transform, glm.vec3(t_x, t_y, t_z))
    
    # aplicando rotacao (segunda operação a ser executada)
    # eixo x
    matrix_transform = glm.rotate(matrix_transform, math.radians(r_x), glm.vec3(1, 0, 0))
    
    # eixo y
    matrix_transform = glm.rotate(matrix_transform, math.radians(r_y), glm.vec3(0, 1, 0))
    
    # eixo z
    matrix_transform = glm.rotate(matrix_transform, math.radians(r_z), glm.vec3(0, 0, 1))
    
    # aplicando escala (primeira operação a ser executada)
    matrix_transform = glm.scale(matrix_transform, glm.vec3(s_x, s_y, s_z))
    
    matrix_transform = np.array(matrix_transform)
    
    return matrix_transform

### Eventos de Teclado

In [11]:
rotacao_fantasma = 169 - 180
escala_lua = 1
caderno_mexendo = False
caderno_amostra = False 
caderno_velocidade = 0
caderno_altura = 0
SHOW_LINES = False

def key_event(window,key,scancode,action,mods):
    global rotacao_fantasma, escala_lua, caderno_mexendo, caderno_velocidade, SHOW_LINES
    
    if key == glfw.KEY_RIGHT: 
        rotacao_fantasma -= 2.0
            
    elif key == glfw.KEY_LEFT: 
        rotacao_fantasma += 2.0
        
    if key == glfw.KEY_UP:
        escala_lua += 0.1
        if escala_lua > 3.0:  # limite opcional
            escala_lua = 3.0

    elif key == glfw.KEY_DOWN:
        escala_lua -= 0.1
        if escala_lua < 0.5:  # limite mínimo
            escala_lua = 0.5
    
    if key == 32 and action == glfw.PRESS and not caderno_mexendo:
        if caderno_amostra:
            caderno_velocidade = -0.1
            caderno_mexendo = True
        else:
            caderno_velocidade = 0.1
            caderno_mexendo = True
    
    if key == glfw.KEY_P and action == glfw.PRESS:
        SHOW_LINES = not SHOW_LINES
        if SHOW_LINES:
            glPolygonMode(GL_FRONT_AND_BACK, GL_LINE)
        else:
            glPolygonMode(GL_FRONT_AND_BACK, GL_FILL)

glfw.set_key_callback(window,key_event)

## Nesse momento, nós exibimos a janela!


In [12]:
glfw.show_window(window)

## Loop principal da janela.

In [13]:
glEnable(GL_DEPTH_TEST) ### importante para 3D
   
while not glfw.window_should_close(window):
    glfw.poll_events() 
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    
    #CÉU
    glClearColor(0, 0, 0.27, 1)

    #objetos 2D

    #GRAMA
    model_objeto(vertice_inicial_grama, num_vertices_grama)
    desenha_objeto(vertice_inicial_grama, num_vertices_grama, cor=[0.231,0.329,0.066,1])

    #LUA
    model_objeto(vertice_inicial_lua, num_vertices_lua, t_x=-3, t_y=4, t_z=6, s_x=escala_lua, s_y=escala_lua)
    #círculo
    desenha_objeto(vertice_inicial_lua, 3*40, cor=[0.764,0.764,0.764,1])
    # cratera 1
    desenha_objeto(vertice_inicial_lua + 3*40, 3*20, cor=[0.5, 0.5, 0.5, 1])
    # cratera 2
    desenha_objeto(vertice_inicial_lua + 3*40 + 3*20, 3*20, cor=[0.5, 0.5, 0.5, 1])
    # cratera 3
    desenha_objeto(vertice_inicial_lua + 3*40 + 3*20*2, 3*20, cor=[0.5, 0.5, 0.5, 1])
    # cratera 4
    desenha_objeto(vertice_inicial_lua + 3*40 + 3*20*3, 3*20, cor=[0.5, 0.5, 0.5, 1])

    #CABANA
    model_objeto(vertice_inicial_cabana, num_vertices_cabana, t_x=0, t_y=1, t_z=4, s_x=1.25, s_y=1.2)
    #parede
    desenha_objeto(vertice_inicial_cabana, 6, cor=[0.411,0.247,0.145,1])
    #porta
    desenha_objeto(vertice_inicial_cabana + 6, 6, cor=[0.6,0.317,0.188,1])
    #teto
    desenha_objeto(vertice_inicial_cabana + 12, num_vertices_cabana-12, cor=[0.219, 0.145, 0.12,1])

    #CADERNO
    model_objeto(vertice_inicial_caderno, num_vertices_caderno, t_x=0, t_y=-7+caderno_altura, t_z=0, s_x=-3, s_y=3, s_z=3)
    desenha_objeto(vertice_inicial_caderno, 2*3, cor=[1,1,1,1])
    desenha_objeto(vertice_inicial_caderno + 2*3, 18*3, cor=[0,0,0,1])
    desenha_objeto(vertice_inicial_caderno + 20*3, 10*3, cor=[0.65,0.85,0.93,1])
    
    # #objetos 3D
    
    # desenhando fantasma
    model_objeto(vertice_inicial_fantasma, num_vertices_fantasma, t_x=-2, t_z=2, r_y=rotacao_fantasma)
    # corpo
    desenha_objeto(vertice_inicial_fantasma, 1320, cor=[0.470,0.427,0.545,1])
    # olho
    desenha_objeto(vertice_inicial_fantasma + 1320, 6,  cor=[1,1,1,1])
    #boca
    desenha_objeto(vertice_inicial_fantasma + 1326, 12,  cor=[0.760,0.529,0.647,1])

    # desenhando arvores
    model_arvores = [
        {'t_x':4.3, 't_y':-2.5, 't_z':0, 's_x':1.6, 's_y':4, 's_z':2.5},
        {'t_x':2, 't_y':-1.5, 't_z':2, 's_x':2.5, 's_y':2.5, 's_z':2.5},
        {'t_x':3.6, 't_y':-2.5, 't_z':1.5, 's_x':2, 's_y':1.5, 's_z':1.8},
        {'t_x':0.7, 't_y':-2.5, 't_z':1.5, 's_x':1.5, 's_y':1, 's_z':1.8},
        {'t_x':-5, 't_y':-2.5, 't_z':1.5, 's_x':1.7, 's_y':4, 's_z':4},
        {'t_x':-3, 't_y':-1.5, 't_z':5, 's_x':1.7, 's_y':2, 's_z':2.5},
        {'t_x':-4, 't_y':-2.2, 't_z':1.8, 's_x':1.5, 's_y':1, 's_z':1.4},
    ]
    for i, arvore in enumerate(arvores):
        model_objeto(arvore['vertice_inicial'], arvore['num_vertices'], **model_arvores[i])
        ## tronco
        desenha_objeto(vertice_inicial_arvore, 121, cor=[0.219, 0.145, 0.12, 0.35])
        ## copa
        desenha_objeto(vertice_inicial_arvore+120, num_vertices_arvore-120, cor=[0.043, 0.211, 0.09, 0.4])
    
    #movimento caderno
    if caderno_mexendo:
        caderno_altura += caderno_velocidade
        if caderno_altura > 5:
            caderno_altura = 5
            caderno_mexendo = False
            caderno_amostra = True
        if caderno_altura < 0:
            caderno_altura = 0
            caderno_mexendo = False
            caderno_amostra = False
    
    glfw.swap_buffers(window)

glfw.terminate()